<a href="https://colab.research.google.com/github/GeraudBourdin/llm-scripts/blob/main/tester_resultat_autotrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet pandas autotrain-advanced huggingface_hub accelerate
!pip install -q bitsandbytes datasets loralib transformers peft


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

**Test inference 1**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

model_path = 'meta-llama/Llama-2-7b-chat-hf'   # this is the base model, the one you started from and then finetuned
adapter_path = 'Bourdin/autotrain-lty2a-ymnng'  # this is the path to the local dir where autotune saved your adapter

base_model = AutoModelForCausalLM.from_pretrained(
      model_path,  # this is the base model, the one you started from and then finetuned
      low_cpu_mem_usage=True,
      trust_remote_code=True,
      load_in_4bit=True,
      torch_dtype=torch.float16,
      device_map="auto",
  )

model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map="auto",
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(
      adapter_path,
      trust_remote_code=True,
  )

## Next, we want to generate some text. See https://huggingface.co/bertin-project/bertin-alpaca-lora-7b/discussions/1 for why we **can't** use a `pipeline` here

from transformers import GenerationConfig
import textwrap

def pipe(prompt_text):
  input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to('cuda')
  generation_output = model.generate(
          input_ids=input_ids,
          generation_config=GenerationConfig(
              # these params you'll have to tweak
              do_sample=True,
              temperature=0.3,
              top_p=0.9,
              num_beams=2,
              ),
          return_dict_in_generate=True,
          output_scores=True,
          max_new_tokens=150
      )
  for seq in generation_output.sequences:
      output = tokenizer.decode(seq)
      wrapped_string = textwrap.fill(output, width=80)
      print(f'---\n{wrapped_string}')


pipe("""Question test ?""")


**Test inference 2**

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Bourdin/autotrain-lty2a-ymnng"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
batch = tokenizer("“Question test  ?” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))